<a href="https://colab.research.google.com/github/verach3n/DD2360HT22/blob/main/ex_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Wed Nov 30 14:08:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Benchmark: LU Decomposition**
## 1) lud_cuda
Changes in makefile(./lud/cuda/Makefile):

Tesla T4 has the Turing architecture, so line 11 should be `-arch=sm_75 \`    https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/

In [29]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lud
base  common  cuda  Makefile  README  run  tools


In [30]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=64"

cd cuda; make clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lud/cuda'
rm -f lud_cuda ../common/common.o lud.o lud_kernel.o *.linkinfo
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lud/cuda'
cd cuda; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lud/cuda'
nvcc -DRD_WG_SIZE_0=64 -I../common -O3 -use_fast_math -arch=sm_75 -lm  -DGPU_TIMER  -o ../common/common.o -c ../common/common.c
../common/common.c: In function ‘create_matrix_from_file’:
../common/common.c:51:3: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
   fscanf(fp, "%d\n", &size);
   ^~~~~~~~~~~~~~~~~~~~~~~~~
../common/common.c:61:11: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
           fscanf(fp, "%f ", m+i*size+j);
           ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
nvcc -D

In [31]:
!ls
!./cuda/lud_cuda -s 1500

base  common  cuda  Makefile  README  run  tools
WG size of kernel = 64 X 64
Generate input matrix internally, size =1500
Creating matrix internally size=1500
Time consumed(ms): 31.677000


## 2) lud_omp
Changes in makefile(./lud/omp/Makefile.offload):

Line 18 and line 19 should be `-CC              := gcc
CXX             := gcc
`  

In [32]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud
base  common  Makefile	omp  README  run  run_offload  tools


In [33]:
!make clean
!make

cd omp; make clean ; make -f Makefile.offload clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud/omp'
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp_offload lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud/omp'
cd omp; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lud/omp'
gcc  -c -Wall -O3   -fopenmp -I../common lud.c -o lud.o
gcc  -c -Wall -O3   -fopenmp -I../common lud_omp.c -o lud_omp.o
gcc  -c -Wall -O3   -fopenmp -I../common ../common/common.c -o ../common/common.o
../common/common.c: In function ‘create_matrix_from_file’:
../common/common.c:51:3: warning: ignor

In [34]:
!ls
!./omp/lud_omp -s 1500

base  common  Makefile	omp  README  run  run_offload  tools
Generate input matrix internally, size =1500
Creating matrix internally size=1500
running OMP on host
Time consumed(ms): 292.581000


# **2. Benchmark: LavaMD2**
## 1) lavaMD_cuda
Changes in makefile(./lavaMD/cuda/Makefile):

Line 21 should be`CUDA_FLAG = -arch sm_75`

In [62]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/lavaMD
kernel	lavaMD	main.c	main.h	main.o	makefile  README  result.txt  run  util


In [63]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=64"

rm	*.o \
	./kernel/*.o \
	./util/num/*.o \
	./util/timer/*.o \
	./util/device/*.o \
	lavaMD
gcc	-DRD_WG_SIZE_0=64  main.c \
		-c \
		-o main.o \
		-O3
main.c: In function ‘main’:
main.c:95:6: warning: implicit declaration of function ‘strcmp’ [-Wimplicit-function-declaration]
   if(strcmp(argv[dim_cpu.cur_arg], "-boxes1d")==0){
      ^~~~~~
main.c:228:8: warning: implicit declaration of function ‘time’; did you mean ‘true’? [-Wimplicit-function-declaration]
  srand(time(NULL));
        ^~~~
        true
/usr/local/cuda/bin/nvcc -DRD_WG_SIZE_0=64						./kernel/kernel_gpu_cuda_wrapper.cu \
								-c \
								-o ./kernel/kernel_gpu_cuda_wrapper.o \
								-O3 \
								-arch sm_75
./kernel/kernel_gpu_cuda_wrapper.cu: In function ‘void kernel_gpu_cuda_wrapper(par_str, dim_str, box_str*, FOUR_VECTOR*, double*, FOUR_VECTOR*)’:
./kernel/kernel_gpu_cuda_wrapper.cu:66:23: warning: ‘cudaError_t cudaThreadSynchronize()’ is deprecated [-Wdeprecated-declarations]
  cudaThreadSynchronize();
     

In [64]:
!./lavaMD -boxes1d 10

thread block size of kernel = 64 
Configuration used: boxes1d = 10
Time spent in different stages of GPU_CUDA KERNEL:
 0.218876004219 s, 50.504173278809 % : GPU: SET DEVICE / DRIVER INIT
 0.000292000012 s,  0.067377053201 % : GPU MEM: ALO
 0.001889999956 s,  0.436104893684 % : GPU MEM: COPY IN
 0.211007997394 s, 48.688686370850 % : GPU: KERNEL
 0.000738999981 s,  0.170519307256 % : GPU MEM: COPY OUT
 0.000577000028 s,  0.133138895035 % : GPU MEM: FRE
Total time:
0.433382004499 s


## 2) lavaMD_omp
No necenssary changes in makefile.

In [58]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/lavaMD
kernel	main.c	main.h	makefile  README  run  util


In [59]:
!make clean
!make OUTPUT=Y

rm	*.o \
	./kernel/*.o \
	./util/num/*.o \
	./util/timer/*.o \
                ./util/device/*.o \
	lavaMD
rm: cannot remove '*.o': No such file or directory
rm: cannot remove './kernel/*.o': No such file or directory
rm: cannot remove './util/num/*.o': No such file or directory
rm: cannot remove './util/timer/*.o': No such file or directory
rm: cannot remove './util/device/*.o': No such file or directory
rm: cannot remove 'lavaMD': No such file or directory
makefile:67: recipe for target 'clean' failed
make: *** [clean] Error 1
gcc	-DOUTPUT main.c \
		-c \
		-o main.o \
		-O3
main.c: In function ‘main’:
main.c:95:6: warning: implicit declaration of function ‘strcmp’ [-Wimplicit-function-declaration]
   if(strcmp(argv[dim_cpu.cur_arg], "-cores")==0){
      ^~~~~~
main.c:253:8: warning: implicit declaration of function ‘time’; did you mean ‘true’? [-Wimplicit-function-declaration]
  srand(time(NULL));
        ^~~~
        true
gcc				./kernel/kernel_cpu.c \
					-c \
					-o ./kernel/ke

In [61]:
!./lavaMD -cores 4 -boxes1d 10

Configuration used: cores = 4, boxes1d = 10
Time spent in different stages of CPU/MCPU KERNEL:
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: VARIABLES
 0.000010000000 s,  0.000162096127 % : MCPU: SET DEVICE
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: INPUTS
 6.169168949127 s, 99.999839782715 % : CPU/MCPU: KERNEL
Total time:
6.169178962708 s


# **3. Benchmark: B+ Tree**
## 1) b+tree_cuda
Changes in makefile(./b+tree/cuda/Makefile):

Tesla T4 has the Turing architecture, so line 4 should be `CUDA_FLAG = -arch sm_75`    https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/

In [100]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/b+tree'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/cuda/b+tree
b+tree.out  kernel  main.h  Makefile	penmp	run
common.h    main.c  main.o  output.txt	README	util


In [101]:
!make clean
!make KERNEL_DIM="-DRD_WG_SIZE_0=64"

rm	*.o *.out \
	./kernel/*.o \
	./util/timer/*.o \
	./util/num/*.o \
	./util/cuda/*.o \
                output.txt
gcc	-DRD_WG_SIZE_0=64 ./main.c \
		-c \
		-o ./main.o \
		-O3
./main.c: In function ‘transform_to_cuda’:
./main.c:652:2: warning: implicit declaration of function ‘gettimeofday’; did you mean ‘get_time’? [-Wimplicit-function-declaration]
  gettimeofday (&one, NULL);
  ^~~~~~~~~~~~
  get_time
./main.c:737:34: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
   printf("\nNumber of records = %d, sizeof(record)=%d, total=%d\n",size,sizeof(record),size*sizeof(record));
                                 ~^
                                 %ld
./main.c:737:53: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘long unsigned int’ [-Wformat=]
   printf("\nNumber of records = %d, sizeof(record)=%d, total=%d\n",size,sizeof(record),size*sizeof(record));
                                                    ~^
 

In [102]:
!nvprof ./b+tree.out file ../../data/b+tree/mil.txt command ../../data/b+tree/command.txt

WG size of kernel 1 & 2  = 64 
==3580== NVPROF is profiling process 3580, command: ./b+tree.out file ../../data/b+tree/mil.txt command ../../data/b+tree/command.txt
Selecting device 0
Input File: ../../data/b+tree/mil.txt 
Command File: ../../data/b+tree/command.txt 
Command Buffer: 
j 6000 3000
k 10000


Getting input from file ../../data/b+tree/mil.txt...
Transforming data to a GPU suitable structure...
Tree transformation took 1.510372
Waiting for command
> 
******command: j count=6000, rSize=6000 
knodes_elem=32257, knodes_unit_mem=532, knodes_mem=17160724
# of blocks = 6000, # of threads/block = 64 (ensure that device can handle)
Time spent in different stages of GPU_CUDA KERNEL:
 0.310860008001 s, 98.328918457031 % : GPU: SET DEVICE / DRIVER INIT
 0.000288999989 s,  0.091414324939 % : GPU MEM: ALO
 0.003872999921 s,  1.225078463554 % : GPU MEM: COPY IN
 0.000148000006 s,  0.046814259142 % : GPU: KERNEL
 0.000043000000 s,  0.013601439074 % : GPU MEM: COPY OUT
 0.000929999980 s,  0

## 2) b+tree_omp
No necessary changes in makefile.

In [103]:
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/b+tree'
!ls

/content/drive/MyDrive/Colab Notebooks/DD2360/rodinia_3.1/openmp/b+tree
common.h  kernel  main.c  main.h  Makefile  penmp  run	util


In [104]:
!make clean
!make

rm	*.o *.out \
	./kernel/*.o \
	./util/timer/*.o \
	./util/num/*.o \
                output.txt
rm: cannot remove '*.o': No such file or directory
rm: cannot remove '*.out': No such file or directory
rm: cannot remove './kernel/*.o': No such file or directory
rm: cannot remove './util/timer/*.o': No such file or directory
rm: cannot remove './util/num/*.o': No such file or directory
rm: cannot remove 'output.txt': No such file or directory
Makefile:89: recipe for target 'clean' failed
make: *** [clean] Error 1
gcc	./main.c \
		-c \
		-o ./main.o \
		-O3
./main.c: In function ‘transform_to_cuda’:
./main.c:652:2: warning: implicit declaration of function ‘gettimeofday’; did you mean ‘get_time’? [-Wimplicit-function-declaration]
  gettimeofday (&one, NULL);
  ^~~~~~~~~~~~
  get_time
./main.c:737:34: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
   printf("\nNumber of records = %d, sizeof(record)=%d, total=%d\n",size,sizeof(record),size

In [105]:
!nvprof ./b+tree.out core 2 file ../../data/b+tree/mil.txt command ../../data/b+tree/command.txt

Input File: ../../data/b+tree/mil.txt 
Command File: ../../data/b+tree/command.txt 
Command Buffer: 
j 6000 3000
k 10000


Getting input from file core...
Transforming data to a GPU suitable structure...
Tree transformation took 0.030762
Waiting for command
> 
******command: j count=6000, rSize=6000 
Time spent in different stages of CPU/MCPU KERNEL:
 0.000015000000 s,  0.103906899691 % : MCPU: SET DEVICE
 0.014421000145 s, 99.896087646484 % : CPU/MCPU: KERNEL
Total time:
0.014436000027 s
> > > > > > > > > > > > 
 ******command: k count=10000 
Time spent in different stages of CPU/MCPU KERNEL:
 0.000000000000 s,  0.000000000000 % : MCPU: SET DEVICE
 0.014205999672 s, 100.000000000000 % : CPU/MCPU: KERNEL
Total time:
0.014205999672 s
> > > > > > > > > > 
======== Warning: No profile data collected.
